In [47]:
import os
import re
import csv
import glob
import pandas as pd
from enum import Enum

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

column_names = ['CURRENT_SAC_END_TIME', 'CURRENT_SAC_AVG_VELOCITY', 'CURRENT_SAC_PEAK_VELOCITY', 'CURRENT_SAC_DURATION']
column_names_fix = ['CURRENT_FIX_END', 'CURRENT_FIX_DURATION']
column_dict = {}
skip_val = ['.']

step = 30000

Health = ["Без заикания", "До лечения", "Во время лечения"]

def dists(v):
    mean = np.mean(v)
    return [np.linalg.norm(x - mean) for x in v]

class EyeDat:

    """ The class that handles the dataset """
    def __init__(self, health, exp, trial):
        self.health = health
        self.exp = exp
        patients = os.listdir("./Данные айтрекера/{}/".format(health))
        fnames = [(glob.glob("./*/{}/{}/Fix*_{}*".format(health, pat, exp))[0], glob.glob("./*/{}/{}/Simple*_{}*".format(health, pat, exp))[0]) for pat in patients]
        df_s = [(pd.read_csv(n[0], delim_whitespace =1, decimal=',',usecols=["CURRENT_FIX_START","CURRENT_FIX_END","CURRENT_FIX_X","CURRENT_FIX_Y","CURRENT_FIX_DURATION"]),
                 pd.read_csv(n[1], delim_whitespace =1, decimal=',',usecols=["LEFT_GAZE_X","LEFT_GAZE_Y","RIGHT_GAZE_X","RIGHT_GAZE_Y"])) for n in fnames]
        lst = []
        disps = self.get_disps(df_s)
        df = pd.concat([pd.concat(df_s[0]),pd.Series(disps)],axis=1)
        self.df = df

    def get_disps(self,df_s):
        disps = []
        for fixs,simp in df_s:
            df = simp.loc[:,(simp != '.').any(axis=0)]
            df = df.loc[:, (df != '.').any(axis=0)]
            df = df.stack().str.replace(',','.').unstack()
            df = df[df[df.columns[0]]!='.'].astype(float)
            simp = df
            step = len(simp)/len(fixs)
            for rec in fixs[["CURRENT_FIX_START","CURRENT_FIX_END"]].itertuples():
                fixation = np.asarray(simp[int(step*rec[1]):int(step*rec[2])])
                disps.append(dists(fixation))

        return disps

    def hist_calc(self, axes):
#         mean = np.array((self.df["CURRENT_FIX_X"].mean(), self.df["CURRENT_FIX_Y"].mean()))
#         pairs = ([np.linalg.norm(x - mean) for x in np.array((self.df["CURRENT_FIX_X"],self.df["CURRENT_FIX_Y"])).transpose()],self.df["CURRENT_FIX_DURATION"])
        pairs = (self.disps, self.df["CURRENT_FIX_DURATION"])
        axes.scatter(pairs[0],pairs[1], s = 4)
        axes.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        if self.exp == 1 : axes.set_ylabel(self.health)
        if self.health == "Во время лечения" : axes.set_xlabel("Эксперимент "+ str(self.exp))
        return pairs


In [41]:
patients = os.listdir("./Данные айтрекера/{}/".format(Health[0]))
fnames = [(glob.glob("./*/{}/{}/Fix*_{}*".format(Health[0],pat , 4))[0], glob.glob("./*/{}/{}/Simple*_{}*".format(Health[0], pat, 4))[0]) for pat in patients]
fnames[0][0]

'./Данные айтрекера/Без заикания/633 Новожилов/Fix_Nov_4.xls'

In [22]:
from fix_scatter import *

In [48]:
EyeDat(Health[0],1,1)

/home/a.shedko/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/a.shedko/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [46]:
pd.Series([1,1,1,1])

0    1
1    1
2    1
3    1
dtype: int64